## **Libraries and Dependencies**

In [ ]:
# Imports
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, optimizers, applications
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import RandomZoom, RandomRotation, RandomFlip, Rescaling, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from PIL import Image
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler
from tensorflow.keras.applications import VGG16
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import pandas as pd
import cv2
import logging
import warnings
import json


# Suppress warnings from the logging module
logging.getLogger('tensorflow').setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=UserWarning)


## **Tensorflow Version**

In [ ]:
# Tensorflow Version
print(tf.__version__)

## **GPU Checker**

In [ ]:
# Check if any GPU devices are detected
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPUs detected: {len(gpus)}")
else:
    print("No GPU detected.")

## **Tensorflow Warning Suppression**

In [ ]:
# Suppress TensorFlow logging except for fatal errors.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

## **Global Variables**

In [ ]:
# Set seed
SEED = 338424

# Global variables
IMG_SIZE = (64, 64)
BATCH_SIZE = 32
num_classes = 18 # Number of folders in dataset
AUTOTUNE = tf.data.AUTOTUNE

## **Dataset: Loading, Splitting, Shuffling, Caching**

In [ ]:
# Load Dataset
dataset_dir = 'dataset/hagridset'
full_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    shuffle=True,
    seed=SEED,
    image_size=(IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

# Split into training, validation, and test sets
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# Total length of the dataset
total_size = len(full_ds)

# Compute indices for the splits
train_size = int(total_size * train_ratio)
val_size = int(total_size * val_ratio)
test_size = total_size - (train_size + val_size)

# Split the dataset and shuffle
train_ds = full_ds.take(train_size).shuffle(train_size, seed=SEED)
val_ds = full_ds.skip(train_size).take(val_size).shuffle(val_size, seed=SEED)
test_ds = full_ds.skip(train_size + val_size).shuffle(test_size, seed=SEED)

# Cache the dataset in memory (or use a directory to store it on disk if necessary)
train_ds = full_ds.take(train_size).shuffle(train_size, seed=SEED).cache().prefetch(buffer_size=AUTOTUNE)
val_ds = full_ds.skip(train_size).take(val_size).shuffle(val_size, seed=SEED).cache().prefetch(buffer_size=AUTOTUNE)
test_ds = full_ds.skip(train_size + val_size).cache().prefetch(buffer_size=AUTOTUNE)

# Count samples in each subset
def count_samples(dataset):
    sample_count = sum(1 for _ in dataset.unbatch())
    return sample_count

# Output the number of samples for each dataset
print(f'Using {count_samples(train_ds)} samples in the Training set')
print(f'Using {count_samples(val_ds)} samples in the Validation set')
print(f'Using {count_samples(test_ds)} samples in the Test set')

In [ ]:
# Get class names
class_names = full_ds.class_names
class_names

## **Regularization Factors**

This code snippet defines the values for L1 and L2 regularization, which are both set to 0.01. It then creates an "Elastic Net Regularizer" that combines these L1 and L2 values to help prevent the model from overfitting by penalizing overly complex or large weight values in the model's learning process.

In [ ]:
# Define L1 and L2 regularization factors
l1_factor = 0.01  # Example value
l2_factor = 0.01  # Example value

# Elastic Net Regularizer
elastic_net_regularizer = regularizers.l1_l2(l1=l1_factor, l2=l2_factor)

## **Callbacks: Learning Rate Scheduler and Early Stopping**

In [ ]:
# Define a learning rate schedule
def lr_time_based_decay(epoch, lr):
    # This function adjusts the learning rate over each epoch based on the initial learning rate,
    # applying a decay factor that increases with the epoch number. It effectively reduces the 
    # learning rate over time, which can help in calibrating the model adjustments as it 
    # approaches a minimum in the loss surface.
    return lr * 1 / (1 + 0.01 * epoch)

# Define callbacks
callbacks = [
    # EarlyStopping prevents overfitting by stopping training when the validation loss 
    # has not improved for 3 consecutive epochs ('patience=3'). It also restores the 
    # weights of the model to those of the epoch with the best validation loss, ensuring 
    # the model retains the best learned features even if it starts to overfit afterward.
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    # LearningRateScheduler adjusts the learning rate according to the lr_time_based_decay function above.
    # It logs the new learning rate at the start of each epoch ('verbose=1'), helping to control
    # the step size of model updates, which can be crucial for reaching convergence efficiently.
    LearningRateScheduler(lr_time_based_decay, verbose=1)
]

## **Data Augmentation**

In [ ]:
# Data Augmentation
data_augmentation_layers = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
])

---

# **Convolutional Neural Networks (CNN): Deep Models**

---

## **CNN Model: Deep**